In [124]:
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import time
np.random.seed(4723423)

In [125]:
Ncons          = 1000                                                                                       # Tot. number of consumers per firm
Nmkt           = 1                                                                                         # Number of markets
Nprod          = 20                                                                                        # Number of products
beta           = np.array([3.0,1.0])                                                                           # True value

def create_data(Ncons, Nmkt, Nprod, beta):
    data_mu        = np.random.normal(0,1,7)                                                               # Mean data (A and Z)    
    data_var       = [ [ 0.9920,  -0.3210,    0.2962,    0.3144,    0.5061,   -0.0014,     0.0077],
                     [-0.3210,   0.9996,    0.3459,    0.2765,    0.0062,    0.4638,     0.0413],
                     [ 0.2962,   0.3459,    1.0341,    0.3082,    0.0301,   -0.0101,     0.5034],
                     [ 0.3144,   0.2765,    0.3082,    1.8842,    0.7012,    0.6674,     0.6345],
                     [ 0.5061,   0.0062,    0.0301,    0.7012,    1.0150,    0.1950,     0.2173],
                     [-0.0014,   0.4638,   -0.0101,    0.6674,    0.1950,    0.9788,     0.1860],
                     [ 0.0077,   0.0413,    0.5034,    0.6345,    0.2173,    0.1860,     0.9548] ]          # Var-cov matrix of the data
    IDprod         = np.sum(np.kron(np.identity(Nmkt),np.linspace(1,Nprod,Nprod)),0)                        # IDprod
    IDmkt          = np.sum(np.kron(np.linspace(1,Nmkt,Nmkt),np.identity(Nprod)),0)                         # IDmkt
    X              = np.array(np.random.multivariate_normal(data_mu, data_var, Nprod*Nmkt))                 # Draw matrix of exogenous prod characteristics and instruments
    e              = np.random.gumbel(0,1,(Nprod*Nmkt,Ncons))
    Const          = np.ones(Nprod*Nmkt)
    u              = np.zeros([Nprod*Nmkt,Ncons])
    for i in range(Ncons):
        u[:,i]             = X[:,0]*beta[0] + X[:,1]*beta[1] + e[:,i]
    y              = np.argmax(u,axis=0)                                                                    # Consumer choice
    return([y,X])

In [126]:
def obj_fun(beta):
    prob = np.exp(X[:,0]*beta[0] + X[:,1]*beta[1]) / ( 1.0 +np.sum(np.exp(X[:,0]*beta[0] + X[:,1]*beta[1])) )
    ll   = - np.sum(np.log(prob[y]))
    return(ll)

# NB. cannot identify constant term in the likelihood

In [128]:
Nrep = 100
res  = np.empty([2,Nrep])
prob  = np.empty([2,Nrep])

start             = time.time()

for i in range(Nrep):
    data    = create_data(Ncons, Nmkt, Nprod, beta)
    y       = data[0]
    X       = data[1]
    res[:,i]  = minimize(obj_fun, np.array([0.0,0.0])).x
    #print(obj_fun(res[:,i]))
    
end               = time.time() 
print("time elapsed:", end-start, "seconds")

np.mean(res,1)


time elapsed: 1.5602707862854004 seconds


array([2.95694035, 0.93682894])